In [1]:
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pds
import numpy as np
import glob
import pandas as pd

In [4]:
# Function to extract catchment ID from filename
def extract_catchment_id(filename):
    # Assuming filename format is 'lamahice_X.nc', where X is the catchment ID
    return filename.split('_')[-1].split('.')[0]

# We use the RAV-II data to calculate Reference ET

In [5]:
# Define functions needed
def calculate_dewpoint(temp_k, specific_humidity, pressure):
    # Constants
    a = 17.27
    b = 237.7
    epsilon = 0.622

    # Convert temperature to Celsius
    temp_c = temp_k - 273.15

    # Calculate saturation vapor pressure (E_s) in hPa
    e_s = 6.112 * np.exp((a * temp_c) / (b + temp_c))

    # Calculate actual vapor pressure (E) using specific humidity and pressure
    # Pressure should be in hPa for consistency
    
    # Convert pressure from Pa to hPa for the calculation
    pressure_hpa = pressure / 100
    
    e = (specific_humidity * pressure_hpa) / (epsilon + (1 - epsilon) * specific_humidity)

    # Calculate relative humidity
    rh = 100 * (e / e_s)

    # Calculate gamma(T, RH)
    gamma = (a * temp_c / (b + temp_c)) + np.log(rh / 100)

    # Calculate dewpoint temperature
    dewpoint = (b * gamma) / (a - gamma)

    return dewpoint

In [6]:
# First we read the Surface Pressure that was exported separately
path = Path(r"C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\rav2\Basins_A\rav2_PSFC_daily_Basins_A.csv")
df_psfc = pds.read_csv(path)
df_psfc = df_psfc.set_index('Time')
df_psfc.index = pds.to_datetime(df_psfc.index)
df_psfc_daily = df_psfc.resample('D').mean()

In [9]:
# Now we loop through all IDs and calculate PET and save as .csv
path = Path(r"C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\rav2\Basins_A\csv")
filepaths = glob.glob(str(path) + '/*.csv')

for fpath in filepaths:
    df = pds.read_csv(fpath)
    df = df.set_index('Time')
    df.index = pds.to_datetime(df.index) #['time']
    data = df.resample('D').mean()

    ID = extract_catchment_id(fpath)
    pressure = df_psfc_daily[ID]/1000 # Convert pressure from Pa to kPa

    albedo = 0.23 # Hypothetical crop albedo from FAO
    sw_net = data['SWDOWN'] * (1-albedo)
    lw_net = data['GLW'] - data['OLR']

    # Convert radiation from W/m² (J/s/m²) to MJ/m²/day
    Rn = (sw_net - lw_net) * (1/1000000) * 86400

    G = data['GRDFLX'] * (1/1000000) * 86400 #Rn*0.3

    # Calculate dewpoint temperature
    dT = calculate_dewpoint(data['T2'], data['Q2'], df_psfc_daily[ID])

    # Remaining calculations (assuming dT is in Celsius)
    es = 0.6108 * np.exp(17.27 * (data['T2'] - 273.15) / ((data['T2'] - 273.15) + 237.3))
    ea = 0.6108 * np.exp(17.27 * (dT / (dT + 237.3)))

    delta = 4098 * es / np.power(data['T2'] - 273.15 + 237.3, 2)
    gamma = 0.665 * 10**-3 * pressure # / 1000  # Convert pressure from Pa to kPa

    U10 = np.sqrt(data['U10']**2 + data['V10']**2)
    U2 = U10 * (4.87 / np.log(67.8 * 10 - 5.42))

    # Complete the Penman-Monteith equation
    numerator = 0.408 * delta * (Rn + G) + gamma * (900 / (data['T2'])) * U2 * (es - ea)
    # We add G because flux of heat from ground to atmosphere is positive
    denominator = delta + gamma * (1 + 0.34 * U2)
    pet = numerator / denominator

    # Save the reference ET data
    pet.name='PET'
    pet.index.name = 'date' 
    save_path = Path(r"C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\PET_Calculations\ref_ET_from_RAV_PM\%s.csv" % ID)
    pet.to_csv(save_path) 